## Camera-based Tic-Tac-Toe

### Hello in this notebook I will go through the process of implementing Tic-Tac-Toe game when the input of the user is just the hand position collected from the web cam

### so the expected output is like this : Tic-Tac.mp4 video 


## the main idea is :-
### - The model recognises the right hand pose and its relative position in the frame
### - then some mapping is done to check if the user hand is near certain cell in the board
### - after that the game logic goes normally (3 consecutive symbols mean someone won)

### So Lets start

### Importing libraries

In [41]:
import pygame, sys, random
import cv2
import numpy as np
import os


### the below framework is the most important in this project as it is the responible for hand pose recognition so big THANK YOU to those developers

### please take a look at their repo 

https://google.github.io/mediapipe/getting_started/python

In [42]:
import mediapipe as mp

### Lets define some constants to be used 

In [43]:
# the Game window dimensions
WIDTH = 640
HEIGHT = 600


# the lines of the board dimensions
LINE_WIDTH = 10
WIN_LINE_WIDTH = 15

# Board parameters
BOARD_ROWS = 3
BOARD_COLS = 3
SQUARE_SIZE = 200


# The O params
CIRCLE_RADIUS = 60
CIRCLE_WIDTH = 15
CROSS_WIDTH = 25

SPACE = 55

# some useful RGB colors
RED = (255, 0, 0)
BG_COLOR = (28, 170, 156)
LINE_COLOR = (200, 200, 200)
CIRCLE_COLOR = (239, 231, 200)
CROSS_COLOR = (255, 0, 0)


### Some Functions definitions

In [44]:

def draw_lines():
    ''''
    this function takes no argument
    
    used to draw the horizontal and vertical lines of the board
    
    it uses the constants defined previously 
    
    Recomendations : pass those variables as params to the function
    '''
    # 1 horizontal
    pygame.draw.line( screen, LINE_COLOR, (0, SQUARE_SIZE), (WIDTH, SQUARE_SIZE), LINE_WIDTH )
    # 2 horizontal
    pygame.draw.line( screen, LINE_COLOR, (0, 2 * SQUARE_SIZE), (WIDTH, 2 * SQUARE_SIZE), LINE_WIDTH )

    # 1 vertical
    pygame.draw.line( screen, LINE_COLOR, (SQUARE_SIZE, 0), (SQUARE_SIZE, HEIGHT), LINE_WIDTH )
    # 2 vertical
    pygame.draw.line( screen, LINE_COLOR, (2 * SQUARE_SIZE, 0), (2 * SQUARE_SIZE, HEIGHT), LINE_WIDTH )

    
    
    
    
def draw_figures():
    '''
    this function is responible for drawing the Os or the Crosses
    no args needed
    '''
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 1:
                pygame.draw.circle( screen, CIRCLE_COLOR, (int( col * SQUARE_SIZE + SQUARE_SIZE//2 ), int( row * SQUARE_SIZE + SQUARE_SIZE//2 )), CIRCLE_RADIUS, CIRCLE_WIDTH )
            elif board[row][col] == 2:
                pygame.draw.line( screen, CROSS_COLOR, (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE), (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SPACE), CROSS_WIDTH )	
                pygame.draw.line( screen, CROSS_COLOR, (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SPACE), (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE), CROSS_WIDTH )

                
                
                
                
def mark_square(row, col, player):
    ''' 
    sets the cells of the board matrix when the player chooses a spot to use
    
    row : an integer defines which row of the matrix
    
    col : an integer defines which col of the matrix
    
    player : either 1 or 2 (player 1 or player 2)
    
    '''
    board[row][col] = player

def available_square(row, col):
    # cheks if this spot is empty
    return board[row][col] == 0

def is_board_full():
    ''' 
    checks for full board
    
    returns true if all spots are taken and false otherwise
     
    '''
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 0:
                return False

    return True

def check_win(player):
    # check if the player(passed as a param) wins by any way
    global game_over
    # vertical win check
    for col in range(BOARD_COLS):
        if board[0][col] == player and board[1][col] == player and board[2][col] == player:
            draw_vertical_winning_line(col, player)
            return True

    # horizontal win check
    for row in range(BOARD_ROWS):
        if board[row][0] == player and board[row][1] == player and board[row][2] == player:
            draw_horizontal_winning_line(row, player)
            return True

    # asc diagonal win check
    if board[2][0] == player and board[1][1] == player and board[0][2] == player:
        draw_asc_diagonal(player)
        return True

    # desc diagonal win chek
    if board[0][0] == player and board[1][1] == player and board[2][2] == player:
        draw_desc_diagonal(player)
        return True

    return False




def draw_vertical_winning_line(col, player):
    
    posX = col * SQUARE_SIZE + SQUARE_SIZE//2

    if player == 1:
        color = CIRCLE_COLOR
    elif player == 2:
        color = CROSS_COLOR

    pygame.draw.line( screen, color, (posX, 15), (posX, HEIGHT - 15), LINE_WIDTH )
    pygame.display.update()
    pygame.quit()
    sys.exit()
   


def draw_horizontal_winning_line(row, player):
    posY = row * SQUARE_SIZE + SQUARE_SIZE//2

    if player == 1:
        color = CIRCLE_COLOR
    elif player == 2:
        color = CROSS_COLOR

    pygame.draw.line( screen, color, (15, posY), (WIDTH - 15, posY), WIN_LINE_WIDTH )
    pygame.display.update()
    pygame.quit()
    sys.exit()
    
def draw_asc_diagonal(player):
    if player == 1:
        color = CIRCLE_COLOR
    elif player == 2:
        color = CROSS_COLOR

    pygame.draw.line( screen, color, (15, HEIGHT - 15), (WIDTH - 15, 15), WIN_LINE_WIDTH )
    pygame.display.update()
    pygame.quit()
    sys.exit()

def draw_desc_diagonal(player):
    if player == 1:
        color = CIRCLE_COLOR
    elif player == 2:
        color = CROSS_COLOR

    pygame.draw.line( screen, color, (15, 15), (WIDTH - 15, HEIGHT - 15), WIN_LINE_WIDTH )
    pygame.display.update()
    pygame.quit()
    sys.exit()
    
def restart():
    # restart the game by clearing all cells 
    screen.fill( BG_COLOR )
    draw_lines()
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            board[row][col] = 0

            
            
            
            
            
def get_input(hand_marks , screen_h , screen_w , is_right_hand , thresh):
    '''
    gets the input from the user's right hand
    
    hand_marks : a NormalizedHandMark object that contains the (x,y,z) 
                 coordinates of the right hand
                 
    screen_h : an integer of the screen height
    
    screen_w : an integer of the screen width
    
    is_right_hand : a boolean indicates if this is the user's right hand
    
    thresh : a float representing the threshold after which 
             the user would be considered too near to the spot 
             so we can safely update the board  
    '''
    global player , SQUARE_SIZE
    
    # the x,y,z coordinates of the right hand 
    # please visit the url provided at the top of the notebook to understand more
    x,y,z = hand_marks.x , hand_marks.y , hand_marks.z
    
    # rescaling those coordinates 
    x *= screen_w
    y *= screen_h
    
    # extracting the indices of the board we should update
    clicked_row = int(y // SQUARE_SIZE) 
    clicked_col = int(x // SQUARE_SIZE) 

    # clipping the indices
    if clicked_row >= 3 : clicked_row = 2
    if clicked_row < 0 : clicked_row = 0
    if clicked_col >= 3 : clicked_col = 2
    if clicked_row < 0 : clicked_col = 0
    
    
    # if empty spot and the z coord. too near and right hand we can perform a move
    
    if (available_square( clicked_row, clicked_col ) 
        and abs(z) > thresh 
        and is_right_hand ):
        
        
        mark_square( clicked_row, clicked_col, player )
        
        if check_win( player ):
            game_over = True
        
        # change your turn
        player = player % 2 + 1
        
        
        
        # AI turn which searches linearly for the next empty spot ... so dump :(
        stop =False

        for i in range(3):
            for j in range(3):
                if board[i][j] == 0:
                    mark_square(i,j,player)
                    stop = True
                    if check_win( player ):
                        game_over = True
                    break
            if stop:
                break
        
        player = player % 2 + 1
        
        # update the window
        draw_figures()
        


### The game loop

In [46]:
# controls where the game window should appear
os.environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % (300,50)

# initializes pygame
pygame.init()

# ------
# SCREEN
# ------
screen = pygame.display.set_mode((WIDTH , HEIGHT))
pygame.display.set_caption( 'TIC TAC TOE' )
screen.fill( BG_COLOR )

# -------------
# CONSOLE BOARD
# -------------
board = np.zeros( (BOARD_ROWS, BOARD_COLS) )



# ---------
# VARIABLES
# ---------
player = 1
game_over = False

# --------
# MAINLOOP
# --------

try :
    
    
    # webcam at position at position 0
    cap = cv2.VideoCapture(0)
    
    # holistic model to process the frame and extract the landmarks
    mp_holistic = mp.solutions.holistic

    # the confidence percentage by which the model detect or track landmarks
    
    with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
        while True:
            for event in pygame.event.get():
                
                # close the game if (X - close button) is pressed
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    break
                
                # restart the game if R is pressed
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:
                        restart()
                        player = 1
                        game_over = False

            # capture frames from the webcam
            ret, frame = cap.read()
            
            # resize the frame
            frame = cv2.resize(frame , (WIDTH,HEIGHT))
            
            # some transperency
            frame =  (0.75 * frame).astype(np.uint8)
            
            # convert it to RGB to be processed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            
            # extract the landmarks and store it at results
            results = holistic.process(image)
            
            
            # left hand and right hand landmarks respectively
            h_marks_l = None
            h_marks_r = None

            
            # if right hand landmarks were detected get the user input
            # NOTE : the lines color turns green if detected 
            if results.right_hand_landmarks:
                h_marks_r = results.right_hand_landmarks.landmark
                LINE_COLOR = (0,255,0)
                get_input(h_marks_r[8] , HEIGHT , WIDTH , player==1 , 0.2)
            else:
                LINE_COLOR = (255,255,255)

            

            # flip the webcam because pygame is flipping it
            frame = np.fliplr(image)
            frame = np.rot90(frame)
            
            # attach the webcam to pygame window
            surf = pygame.surfarray.make_surface(frame)
            screen.blit(surf, (0,0))
            draw_lines()
            draw_figures()
            check_win(3-player)


            # update display
            pygame.display.flip()
            pygame.display.update()
            if game_over:
                pygame.quit()
                sys.exit()
                break

        cap.release()
        cv2.destroyAllWindows()
except:
    cap.release()
    cv2.destroyAllWindows()
    

### That's it and thank you 